In [5]:
!apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [6]:
import os
os.getcwd()

'/teamspace/studios/this_studio'

In [7]:
import os
path = os.getcwd() +'/FinalProject/word2vec-kenlm/data/mesivta/'
file = path + 'train.txt'

before downloading and unpacking the KenLM repo.

In [8]:
import platform
import os

if platform.system() == 'Darwin':  # Darwin stands for MacOS
    !curl -L https://kheafield.com/code/kenlm.tar.gz | tar xz
elif platform.system() == 'Linux':
    !wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
else:
    print("Unsupported operating system")

--2024-03-13 16:28:42--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  2.11MB/s    in 0.2s    

2024-03-13 16:28:42 (2.11 MB/s) - written to stdout [491888/491888]



KenLM is written in C++, so we'll make use of `cmake` to build the binaries.

In [9]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

mkdir: cannot create directory ‘kenlm/build’: File exists
build_binary  fragment	       lmplz			     query
count_ngrams  interpolate      phrase_table_vocab	     streaming_example
filter	      kenlm_benchmark  probing_hash_table_benchmark


In [10]:
!pip install pyctcdecode
!pip install kenlm -U


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Great, as we can see, the executable functions have successfully been built under `kenlm/build/bin/`.

KenLM by default computes an *n-gram* with [Kneser-Ney smooting](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing). All text data used to create the *n-gram* is expected to be stored in a text file.
We download our dataset and save it as a `.txt` file.

Now, we just have to run KenLM's `lmplz` command to build our *n-gram*, called `"5gram.arpa"`. As it's relatively common in speech recognition, we build a *5-gram* by passing the `-o 5` parameter.
For more information on the different *n-gram* LM that can be built
with KenLM, one can take a look at the [official website of KenLM](https://kheafield.com/code/kenlm/).

Executing the command below might take a minute or so.

In [11]:
import os

In [12]:

!kenlm/build/bin/lmplz -o 5 <"{file}" > "{path}/5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /teamspace/studios/this_studio/FinalProject/word2vec-kenlm/data/mesivta/train.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100


****************************************************************************************************
Unigram tokens 4989721 types 161274
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1935288 2:1270423680 3:2382044672 4:3811271168 5:5558104064
Statistics:
1 161274 D1=0.592941 D2=1.05761 D3+=1.50101
2 2136990 D1=0.784366 D2=1.17542 D3+=1.47568
3 3902755 D1=0.90099 D2=1.34268 D3+=1.58097
4 4541403 D1=0.956203 D2=1.50116 D3+=1.68527
5 4760756 D1=0.95573 D2=1.51904 D3+=1.65912
Memory estimate for binary LM:
type     MB
probing 327 assuming -p 1.5
probing 389 assuming -r models -p 1.5
trie    162 without quantization
trie     90 assuming -q 8 -b 8 quantization 
trie    142 assuming -a 22 array pointer compression
trie     70 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:1935288 2:34191840 3:78055100 4:108993672 5:133301168
----5---10---15---20---25---30---35---40---45---50---55

Great, we have built a *5-gram* LM! Let's inspect the first couple of lines.

In [13]:
!head -20 {path}5gram.arpa

\data\
ngram 1=161274
ngram 2=2136990
ngram 3=3902755
ngram 4=4541403
ngram 5=4760756

\1-grams:
-6.3179564	<unk>	0
0	<s>	-0.59340286
-2.8985147	</s>	0
-2.965528	וכן	-0.4433093
-3.892803	פרט	-0.2773799
-6.173038	למסולים	-0.10548098
-2.976362	שהיא	-0.44526786
-4.6349635	נעל	-0.17592013
-2.783812	שאין	-0.5445806
-2.757102	לו	-0.46461868
-4.3451552	עקב	-0.15198769
-3.0338998	שאינו	-0.60274816


There is a small problem that 🤗 Transformers will not be happy about later on.
The *5-gram* correctly includes a "Unknown" or `<unk>`, as well as a *begin-of-sentence*, `<s>` token, but no *end-of-sentence*, `</s>` token.
This sadly has to be corrected currently after the build.

We can simply add the *end-of-sentence* token by adding the line `0 </s>  -0.11831701` below the *begin-of-sentence* token and increasing the `ngram 1` count by 1. Because the file has roughly 100 million lines, this command will take *ca.* 2 minutes.

In [14]:
with open(f"{path}5gram.arpa", "r") as read_file, open(f"{path}5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

Let's now inspect the corrected *5-gram*.

In [15]:
!head -20 {path}5gram_correct.arpa

\data\
ngram 1=161275
ngram 2=2136990
ngram 3=3902755
ngram 4=4541403
ngram 5=4760756

\1-grams:
-6.3179564	<unk>	0
0	<s>	-0.59340286
0	</s>	-0.59340286
-2.8985147	</s>	0
-2.965528	וכן	-0.4433093
-3.892803	פרט	-0.2773799
-6.173038	למסולים	-0.10548098
-2.976362	שהיא	-0.44526786
-4.6349635	נעל	-0.17592013
-2.783812	שאין	-0.5445806
-2.757102	לו	-0.46461868
-4.3451552	עקב	-0.15198769


Great, this looks better! We're done at this point and all that is left to do is to correctly integrate the `"ngram"` with [`pyctcdecode`](https://github.com/kensho-technologies/pyctcdecode) and 🤗 Transformers.

## Create the Processor

In [16]:
# from transformers import AutoProcessor

# processor = AutoProcessor.from_pretrained("imvladikon/wav2vec2-xls-r-300m-hebrew")
from transformers import AutoProcessor


RuntimeError: Failed to import transformers.models.auto because of the following error (look up to see its traceback):
cannot import name 'is_g2p_en_available' from 'transformers.utils' (/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/utils/__init__.py)

Next, we extract the vocabulary of its tokenizer as it represents the `"labels"` of `pyctcdecode`'s `BeamSearchDecoder` class.

In [ ]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

The `"labels"` and the previously built `5gram_correct.arpa` file is all that's needed to build the decoder.

In [ ]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path=f"{path}5gram_correct.arpa",
)

Loading the LM will be faster if you build a binary file.
Reading /teamspace/studios/this_studio/FinalProject/word2vec-kenlm/data/mesivta/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


We can safely ignore the warning and all that is left to do now is to wrap the just created `decoder`, together with the processor's `tokenizer` and `feature_extractor` into a `Wav2Vec2ProcessorWithLM` class.

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

We want to directly upload the LM-boosted processor into
the model folder of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv) to have all relevant files in one place.

Let's clone the repo, add the new decoder files and upload them afterward.
First, we need to install `git-lfs`.

In [ ]:
!sudo apt-get install git-lfs tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 43.0 kB of archives.
After this operation, 115 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tree amd64 1.8.0-1 [43.0 kB]
Fetched 43.0 kB in 0s (295 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package tree.
(Reading database ... 82245 files and directories currently installed.)
Preparing to unpack .../tree_1.8.0-1_amd64.deb ...
Unpacking tree (1.8.0-1) ...
Setting up tree (1.8.0-1) ...
Processing triggers for man-db (2.9.1-1) ...


Cloning and uploading of modeling files can be done conveniently with the `huggingface_hub`'s `Repository` class.

More information on how to use the `huggingface_hub` to upload any files, please take a look at the [official docs](https://huggingface.co/docs/hub/how-to-upstream).

In [ ]:
# from huggingface_hub import Repository

# repo = Repository(local_dir="FinalProject/models/ken-lm", clone_from="imvladikon/wav2vec2-xls-r-300m-hebrew")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/imvladikon/wav2vec2-xls-r-300m-hebrew into local empty directory.


Download file pytorch_model.bin:   0%|          | 32.0k/1.18G [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 2.98k/2.98k [00:00<?, ?B/s]

Clean file training_args.bin:  34%|###3      | 1.00k/2.98k [00:00<?, ?B/s]

Download file model.safetensors:   0%|          | 1.40k/1.18G [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

Having cloned `xls-r-300m-sv`, let's save the new processor with LM into it.

In [ ]:
# processor_with_lm.save_pretrained("xls-r-300m-sv")

Let's inspect the local repository. The `tree` command conveniently can also show the size of the different files.

In [ ]:
# !tree -h xls-r-300m-sv/

As can be seen the *5-gram* LM is quite large - it amounts to more than 4 GB.
To reduce the size of the *n-gram* and make loading faster, `kenLM` allows converting `.arpa` files to binary ones using the `build_binary` executable.

Let's make use of it here.

In [ ]:
# !kenlm/build/bin/build_binary xls-r-300m-sv/language_model/5gram_correct.arpa xls-r-300m-sv/language_model/5gram.bin

Great, it worked! Let's remove the `.arpa` file and check the size of the binary *5-gram* LM.

In [ ]:
# !rm xls-r-300m-sv/language_model/5gram_correct.arpa && tree -h xls-r-300m-sv/

Nice, we reduced the *n-gram* by more than half to less than 2GB now. In the final step, let's upload all files.

In [ ]:
# repo.push_to_hub(commit_message="Upload lm-boosted decoder")

In [ ]:
#save the processor to local
processor_with_lm.save_pretrained("/teamspace/studios/this_studio/FinalProject/models/KenLM-Wav2Vec2-Hebrew-Mesivta")